In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir('..')
sys.path.append('src')

In [ ]:
from rasterio.features import rasterize

In [ ]:
import rasterio
import torch
import numpy as np
from pathlib import Path
import pandas as pd
import torchvision
from dataclasses import dataclass
from omegaconf import OmegaConf
from tqdm import tqdm
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib widget 

from data import read_ann, convert_ann

In [ ]:
def start_points(size, split_size, overlap=0):
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points



def splitter(img, crop_w, crop_h):    
    img_h, img_w, *_ = img.shape
    X_points = start_points(img_w, crop_w, 0.5)
    Y_points = start_points(img_h, crop_h, 0.5)
    
    for i in Y_points:
        for j in X_points:
            split = img[i:i+crop_h, j:j+crop_w]
            #cv2.imwrite('{}_{}.{}'.format(name, count, frmt), split)
            yield split, j, i, crop_h, crop_w

In [ ]:
df = pd.read_csv('input/hmib/train.csv')

In [ ]:
imgs_path = Path('input/hmib/train_images/')
ann_path = Path('input/hmib/train_annotations/')
dst = Path('input/preprocessed/test')
dst.mkdir()
(dst / 'images').mkdir()
(dst / 'masks').mkdir()

In [ ]:
#tdf = df[df.organ == 'spleen']
tdf = df

In [ ]:
H, W = 1024, 1024
ch, cw = 512,512
imgs = []
masks = []
for i,row in tqdm(tdf.iterrows(), total=len(tdf)):
    #if i < 10:
    #    continue
    #break
    f = imgs_path / str(row['id'])
    f = f.with_suffix('.tiff')
    fd = rasterio.open(f)
    a = fd.read().transpose(1,2,0)
    fd.close()
    h,w,c = a.shape
    ar = cv2.resize(a, (W, H))
    ar = ar.astype(np.uint8)
    name = f.with_suffix('.png').name
    
    ss = splitter(ar, ch, cw)
    name, ext = name.split('.')
    for ii, (s,*_) in enumerate(ss):
        fixname = f'{name}_{ii}.{ext}'
        cv2.imwrite(str(dst / 'images' / fixname), s)
    
    #cv2.imwrite(str(dst / 'images' / name), ar)
    
    annf = (ann_path / f.name).with_suffix('.json')
    data = read_ann(annf)
    poly = convert_ann(data)
    mask = rasterize([poly], out_shape=(h,w))
    mask = cv2.resize(mask, (W,H))
    
    ss = splitter(mask, ch, cw)
    #name, ext = name.split('.')
    for ii,  (s,*_) in enumerate(ss):
        fixname = f'{name}_{ii}.{ext}'
        cv2.imwrite(str(dst / 'masks' / fixname), s)
    
    #cv2.imwrite(str(dst / 'masks' / name), mask)
    
    
    # masks.append(mask)
    # imgs.append(ar)
    break